# Train a proprioception-tuned CNN

We create a sensor processing model using CNN-based visual encoding finetuned with proprioception.

We create an encoding for the robot starting from a pretrained CNN model. As the feature vector of this is still large (eg 512 * 7 * 7), we reduce this to the encoding with an MLP. 

We finetune the encoding with information from proprioception.  

The sensor processing object associated with the network trained like this is in sensorprocessing/sp_propriotuned_cnn.py

In [1]:
import sys
sys.path.append("..")

from exp_run_config import Config, Experiment
Config.PROJECTNAME = "BerryPicker"

import pathlib
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

from demonstration.demonstration import Demonstration

import sensorprocessing.sp_helper as sp_helper
from sensorprocessing.sp_propriotuned_cnn import VGG19ProprioTunedRegression, ResNetProprioTunedRegression
from robot.al5d_position_controller import RobotPosition

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [2]:
# Dr. Boloni's path
# external_path = pathlib.Path("/Users/lboloni/Documents/Code/_ExternalExp")
print(Config())
external_path = pathlib.Path(Config()["experiment_external"])
# Sahara's path
# external_path = pathlib.Path("/home/sa641631/SaharaBerryPickerData/experiment_data")
external_path.exists()
Config().set_experiment_path(external_path)
Config().copy_experiment("sensorprocessing_propriotuned_cnn")
Config().copy_experiment("robot_al5d")
Config().copy_experiment("demonstration")

Config().create_exprun_variant("sensorprocessing_propriotuned_cnn","resnet50_128", {"epochs": 17}, new_run_name="boo")

***ExpRun**: Loading pointer config file:
	/Users/lboloni/.config/BerryPicker/mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	/Users/lboloni/Google Drive/My Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/settings-szenes.yaml
***ExpRun**: Experiment config path changed to /Users/lboloni/Documents/Develop/Data/BerryPicker-external
***ExpRun**: Experiment sensorprocessing_propriotuned_cnn copied to /Users/lboloni/Documents/Develop/Data/BerryPicker-external/sensorprocessing_propriotuned_cnn
***ExpRun**: Experiment robot_al5d copied to /Users/lboloni/Documents/Develop/Data/BerryPicker-external/robot_al5d
***ExpRun**: Experiment demonstration copied to /Users/lboloni/Documents/Develop/Data/BerryPicker-external/demonstration
***ExpRun**: Configuration for exp/run: sensorprocessing_propriotuned_cnn/resnet50_128 successfully loaded
***ExpRun**: Exp/run variant sensorprocessing_propriotuned_cnn/boo created in /Users/lboloni/Documents/Develop/Data/BerryPicker-ext

In [3]:
# The experiment/run we are going to run: the specified model will be created
experiment = "sensorprocessing_propriotuned_cnn"
# run = "vgg19_128"
run = "resnet50_128"
# run = "vgg19_256"
# run = "resnet50_256"
# run = "boo"
exp = Config().get_experiment(experiment, run)

exp_robot = Config().get_experiment(exp["robot_exp"], exp["robot_run"])

***ExpRun**: Configuration for exp/run: sensorprocessing_propriotuned_cnn/resnet50_128 successfully loaded
***ExpRun**: Configuration for exp/run: robot_al5d/position_controller_00 successfully loaded


### Create regression training data (image to proprioception)
The training data (X, Y) is all the pictures from a demonstration with the corresponding proprioception data. 

In [4]:
def load_images_as_proprioception_training(exp: Experiment, exp_robot: Experiment):
    """Loads the training images specified in the exp/run. Processes them as two tensors as input and target data for proprioception training.
    Caches the processed results into the input and target file specified in the exp/run.

    Remove those files to recalculate
    """
    retval = {}
    proprioception_input_path = pathlib.Path(exp.data_dir(), "proprio_input.pth")
    proprioception_target_path = pathlib.Path(exp.data_dir(), "proprio_target.pth")

    if proprioception_input_path.exists():
        retval["inputs"] = torch.load(proprioception_input_path, weights_only=True)
        retval["targets"] = torch.load(proprioception_target_path, weights_only=True)
    else:
        inputlist = []
        targetlist = []
        transform = sp_helper.get_transform_to_sp(exp)
        for val in exp["training_data"]:
            run, demo_name, camera = val
            #run = val[0]
            #demo_name = val[1]
            #camera = val[2]
            exp_demo = Config().get_experiment("demonstration", run)
            demo = Demonstration(exp_demo, demo_name)
            for i in range(demo.metadata["maxsteps"]):
                sensor_readings, _ = demo.get_image(i, device=device, transform=transform, camera=camera)
                inputlist.append(sensor_readings[0])
                a = demo.get_action(i)
                rp = RobotPosition.from_vector(exp_robot, a)
                anorm = rp.to_normalized_vector(exp_robot)
                targetlist.append(torch.from_numpy(anorm))
        retval["inputs"] = torch.stack(inputlist)
        retval["targets"] = torch.stack(targetlist)
        torch.save(retval["inputs"], proprioception_input_path)
        torch.save(retval["targets"], proprioception_target_path)

    # Separate the training and validation data.
    # We will be shuffling the demonstrations
    length = retval["inputs"].size(0)
    rows = torch.randperm(length)
    shuffled_inputs = retval["inputs"][rows]
    shuffled_targets = retval["targets"][rows]

    training_size = int( length * 0.67 )
    retval["inputs_training"] = shuffled_inputs[1:training_size]
    retval["targets_training"] = shuffled_targets[1:training_size]

    retval["inputs_validation"] = shuffled_inputs[training_size:]
    retval["targets_validation"] = shuffled_targets[training_size:]

    return retval

In [5]:
exp["training_data"]

[['random-both-cameras-video', '2025_03_08__14_15_53', 'dev2']]

In [6]:
tr = load_images_as_proprioception_training(exp, exp_robot)
inputs_training = tr["inputs_training"]
targets_training = tr["targets_training"]
inputs_validation = tr["inputs_validation"]
targets_validation = tr["targets_validation"]

***ExpRun**: Experiment default config /Users/lboloni/Documents/Develop/Data/BerryPicker-external/demonstration/_defaults_demonstration.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: demonstration/random-both-cameras-video successfully loaded


### Create a model that performs proprioception regression

In [7]:

if exp['model'] == 'VGG19ProprioTunedRegression':
    model = VGG19ProprioTunedRegression(exp, device)
elif exp['model'] == 'ResNetProprioTunedRegression':
    model = ResNetProprioTunedRegression(exp, device)
else:
    raise Exception(f"Unknown model {exp['model']}")

if exp['loss'] == 'MSELoss':
    criterion = nn.MSELoss()
elif exp['loss'] == 'L1Loss':
    criterion = nn.L1Loss()

optimizer = optim.Adam(model.parameters(), lr=exp['learning_rate'])

/Users/lboloni/Documents/Develop/VirtualEnvs/BerryPicker/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/lboloni/Documents/Develop/VirtualEnvs/BerryPicker/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
# Create DataLoaders for batching
batch_size = exp['batch_size']
train_dataset = TensorDataset(inputs_training, targets_training)
test_dataset = TensorDataset(inputs_validation, targets_validation)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
def train_and_save_proprioception_model(model, criterion, optimizer, modelfile, device="cpu", epochs=20):
    """Trains and saves the proprioception model
    FIXME: must have parameters etc to investigate alternative models.
    """
    model = model.to(device)
    criterion = criterion.to(device)
    # Training loop
    num_epochs = epochs
    for epoch in tqdm(range(num_epochs)):
        model.train()
        total_loss = 0
        for batch_X, batch_y in train_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            predictions = model.forward(batch_X)
            loss = criterion(predictions, batch_y)
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        if (epoch + 1) % 1 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}')

    # Evaluate the model
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y)
            test_loss += loss.item()

    test_loss /= len(test_loader)
    print(f'Test Loss: {test_loss:.4f}')
    torch.save(model.state_dict(), modelfile)

In [10]:
modelfile = pathlib.Path(
    exp["data_dir"], exp["proprioception_mlp_model_file"])
epochs = exp["epochs"]
if modelfile.exists():
    print("*** Train-Propriotuned-CNN ***: NOT training; model already exists, loading it")
    model.load_state_dict(torch.load(modelfile))
else:
    train_and_save_proprioception_model(model, criterion, optimizer, modelfile, device=device, epochs=epochs)

  2%|▎         | 1/40 [00:28<18:35, 28.59s/it]

Epoch [1/40], Loss: 8.5426


  5%|▌         | 2/40 [00:57<18:06, 28.60s/it]

Epoch [2/40], Loss: 6.9249


  8%|▊         | 3/40 [01:26<17:51, 28.95s/it]

...

 98%|█████████▊| 39/40 [21:19<00:36, 36.12s/it]

Epoch [39/40], Loss: 0.9380


100%|██████████| 40/40 [21:53<00:00, 32.84s/it]

Epoch [40/40], Loss: 0.7942


Test Loss: 1.1093
